## GCP

# Módulo práctico: IA Generativa en Google Cloud Platform

En este módulo práctico vas a trabajar con las capacidades de **Inteligencia Artificial Generativa** de **Google Cloud Platform (GCP)**, utilizando modelos para procesamiento de texto.

## Tendrás acceso a:

- Modelos LLM de la familia **Gemini**, capaces de comprender y generar texto, analizar imágenes y mantener contexto en conversaciones.  
- La **API oficial de Google Generative AI (Gemini API)**.

## Objetivos del módulo:

- Dominar el uso de los principales modelos LLM de **Gemini** y entender sus diferencias.  
- Construir y analizar consultas.  
- Evaluar y comparar respuestas de distintos modelos y configuraciones.  
- Reflexionar sobre la calidad, ética y limitaciones actuales de la IA en producción.


In [1]:
!pip freeze

annotated-types==0.7.0
anyio==4.11.0
asttokens==3.0.1
beautifulsoup4==4.14.2
cachetools==6.2.2
certifi==2025.11.12
charset-normalizer==3.4.4
colorama==0.4.6
comm==0.2.3
debugpy==1.8.17
decorator==5.2.1
executing==2.2.1
google==3.0.0
google-ai-generativelanguage==0.6.15
google-api-core==2.28.1
google-api-python-client==2.187.0
google-auth==2.43.0
google-auth-httplib2==0.2.1
google-cloud==0.34.0
google-genai==1.50.1
google-generativeai==0.8.5
googleapis-common-protos==1.72.0
grpcio==1.76.0
grpcio-status==1.71.2
h11==0.16.0
httpcore==1.0.9
httplib2==0.31.0
httpx==0.28.1
idna==3.11
ipykernel==7.1.0
ipython==9.7.0
ipython_pygments_lexers==1.1.1
jedi==0.19.2
jupyter_client==8.6.3
jupyter_core==5.9.1
matplotlib-inline==0.2.1
nest-asyncio==1.6.0
packaging==25.0
parso==0.8.5
platformdirs==4.5.0
prompt_toolkit==3.0.52
proto-plus==1.26.1
protobuf==5.29.5
psutil==7.1.3
pure_eval==0.2.3
pyasn1==0.6.1
pyasn1_modules==0.4.2
pydantic==2.12.4
pydantic_core==2.41.5
Pygments==2.19.2
pyparsing==3.2.5
pyth

In [2]:
from google import genai
import os, getpass

In [3]:
# Obtener la API Key de entorno o pedirla al usuario de forma segura
API_KEY = os.getenv("GOOGLE_API_KEY") or getpass.getpass("Introduce tu API Key de Google Generative AI: ")

# Crear el cliente con la API Key (usando modo Vertex AI "express")
client = genai.Client(api_key=API_KEY, vertexai=True)


## Generación simple variando `temperature` y `top_p`

Los modelos generativos permiten ajustar parámetros de muestreo para controlar la aleatoriedad y diversidad de las respuestas.  
Los dos parámetros más comunes son **temperature** (temperatura) y **top_p** (núcleo de probabilidad):

### 🔹 Temperature (`temperature`)

Controla la aletoriedad de elección de la siguiente palabra:

- Un valor bajo (por ejemplo, `0.2`) hace que el modelo sea más conservador y repetitivo.  
- Un valor alto (por ejemplo, `0.8`) lo hace más creativo o impredecible.  
- Una temperatura de `0` significa elegir siempre el token más probable (comportamiento casi determinista).

### 🔹 Top-p (`top_p`)

Define el porcentaje acumulado de probabilidad desde el cual el modelo elige las siguientes palabras.

- Por ejemplo, con `top_p = 0.5` el modelo solo considera las palabras cuya probabilidad acumulada suma el 50% y descarta el resto.  
- Un valor bajo limita la variedad (respuestas más seguras y predecibles).  
- Un valor cercano a `1` considera un rango más amplio de opciones, incrementando la diversidad.

---

Veamos un ejemplo sencillo variando estos parámetros.  
Usaremos el mismo prompt con distintas configuraciones de `temperature` y `top_p` para observar cómo cambia la respuesta.


## Manejo de bloqueos por filtros de seguridad

Google Generative AI incorpora **filtros de seguridad** que pueden bloquear ciertas solicitudes o respuestas consideradas dañinas o inapropiadas.
Es importante manejar estos casos para evitar que nuestra aplicación falle inesperadamente y para respetar las políticas de uso.

---

### ¿Cuándo se bloquea una respuesta?

Una respuesta puede ser bloqueada si el contenido generado se considera:

* Ofensivo
* De incitación al odio
* De violencia extrema
* O de cualquier otra categoría insegura

Cuando esto ocurre:

* El modelo **no devuelve texto**,
* Sino que indica un motivo de finalización especial (`finishReason`).

Por ejemplo, si la respuesta fue filtrada por contenido inseguro, vendrá con:

```json
finishReason: "SAFETY"
```

y sin contenido generado.

Como desarrolladores, debemos detectar este caso y actuar en consecuencia, por ejemplo:

* Mostrar un mensaje de advertencia al usuario.
* Evitar mostrar una respuesta vacía.
* Registrar el evento en logs para análisis futuros.

---

### Configuración de filtros con `safety_settings`

En el SDK, se pueden personalizar los filtros de seguridad a través de `safety_settings` dentro de la configuración del modelo.

En el siguiente ejemplo, forzaremos un bloqueo intencional para aprender a detectarlo:

* Pediremos al modelo que genere lenguaje hostil.
* Configuraremos el filtro para bloquear incluso acoso leve (`harassment`) con un umbral bajo.

Esto nos permitirá observar cómo responde el sistema cuando el filtro de seguridad se activa.


## Salida en formato JSON estructurado

A veces nos interesa que el modelo devuelva datos estructurados (por ejemplo, en formato **JSON**) en lugar de texto libre, para facilitar su procesamiento automático.

Los modelos **Gemini** pueden adaptar sus respuestas a un esquema JSON dado, lo que garantiza una salida con formato predecible y fácil de parsear.  
Esto es especialmente útil para tareas como:

- Extracción de información
- Clasificación estructurada
- Integración con otras herramientas

---

### Uso de JSON Schema con el SDK

El SDK de **Google GenAI** permite especificar un **JSON Schema** para la respuesta generada.

Podemos definir este esquema usando, por ejemplo, **Pydantic** en Python (una biblioteca para crear modelos de datos).

El flujo general es:

1. Definimos una clase en Pydantic con los campos que queremos.
2. Se la pasamos al modelo como requisito de formato.
3. El modelo genera un JSON siguiendo ese esquema.
4. Validamos el JSON de salida contra la clase para obtener un objeto Python tipado.

---

### Ejemplo de uso

Supongamos que queremos extraer información de una frase sobre una persona.

Definiremos un esquema con los siguientes campos:

- `nombre`
- `profesión`
- `edad`
- `país`

Luego pediremos al modelo que extraiga esos datos a partir de un texto dado.


## Uso de sesiones de chat con persistencia de contexto

Además de generar texto a partir de un prompt aislado, el SDK soporta sesiones de chat que mantienen el contexto entre turnos, similar a conversar con ChatGPT u otros asistentes. Esto es útil para diálogos multi-turno donde el modelo debe recordar lo dicho anteriormente y responder acorde.

Con `genai.Client`, podemos crear una sesión de chat y luego enviar mensajes secuencialmente. El contexto (historial de mensajes) se conserva en el objeto de chat, así que el modelo recibe de forma implícita lo que se habló antes.

Veamos un ejemplo:


## Chain-of-Thought prompting (respuesta directa vs paso a paso)

El término **Chain-of-Thought** (Cadena de Pensamiento) se refiere a una técnica de *prompting* donde animamos al modelo a **pensar paso a paso** antes de dar una respuesta final.

En lugar de responder directamente, el modelo expone su **razonamiento intermedio**, lo cual a menudo conduce a respuestas más precisas en problemas complejos.

Podemos lograr esto agregando indicaciones en el prompt del estilo:

- *"Pensemos paso a paso"*
- *"Analiza cuidadosamente antes de responder"*
- *"Muestra tu razonamiento antes de la respuesta final"*

Para ilustrar la diferencia, consideremos una pregunta tipo acertijo o de lógica.  
Haremos que el modelo responda:

1. De forma normal (respuesta directa).  
2. Con una indicación de **cadena de pensamiento** (*Chain-of-Thought*).


# Consulta con múltiples candidatos en Gemini API

Realiza una consulta a un modelo Gemini configurando el parámetro `candidate_count` para obtener varias respuestas alternativas a la vez (por ejemplo, 2 o 3).  
- Elige un prompt de tu interés.
- Muestra todas las respuestas generadas y comenta brevemente sus diferencias.


# Chat y seguimiento de contexto con Gemini API

Vas a crear una simulación de chat entre un usuario y un modelo de Gemini, utilizando la interfaz de chat de la API.

## Instrucciones

1. Inicializa una sesión de chat con un modelo Gemini (elige un modelo que soporte conversación, como `gemini-2.0-flash` o `gemini-2.5-flash`).
2. Realiza **tres interacciones** consecutivas, donde cada mensaje del usuario depende del anterior (por ejemplo, pide primero información general, luego una aclaración o un ejemplo, y finalmente una petición concreta relacionada con los mensajes previos).
3. En la **tercera interacción**, plantea una pregunta que obligue al modelo a referirse explícitamente al contexto o detalles de la conversación anterior (por ejemplo, “¿Puedes resumir lo que hemos hablado hasta ahora?” o “Basándote en lo que me dijiste antes, ¿qué recomendarías?”).
4. Muestra el historial completo del chat y las respuestas del modelo.

## Reflexión

- ¿El modelo fue capaz de mantener el contexto y dar respuestas coherentes?
- ¿Observas alguna limitación o pérdida de información entre turnos?
- ¿Qué técnicas o configuraciones podrían mejorar la memoria conversacional del modelo?
